In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [ ]:
#How to do max pool without the number of points?
#Not sure if I used the TransformerEncoder correctly
# when I call it in forward it says mask takes a tensor, not sure what to use there


# Point Net Backbone
class PointNetBackbone(nn.Module):
    def __init__(self):

        super(PointNetBackbone, self).__init__()
        # transformer encoders
        self.tencoder1 = nn.TransformerEncoder(512, 4)
        self.tencoder2 = nn.TransformerEncoder(512, 4)

        # shared MLP 1
        self.conv1 = nn.Conv1d(3, 128, kernel_size=1)
        self.conv2 = nn.Conv1d(128, 64, kernel_size=1)

        # shared MLP 2
        self.conv3 = nn.Conv1d(64, 512, kernel_size=1)
        self.conv4 = nn.Conv1d(64, 1024, kernel_size=1)

        # batch norms for both shared MLPs
        self.bn1 = nn.BatchNorm1d(128)
        self.bn2 = nn.BatchNorm1d(64)
        self.bn3 = nn.BatchNorm1d(512)
        self.bn4 = nn.BatchNorm1d(1024)

        # max pool to get the global features
        self.max_pool = nn.MaxPool1d(kernel_size=num_points)


    def forward(self, x):

        # get batch size
        bs = x.shape[0]

        # pass through first encoder layer
        x = self.tencoder1(x, mask=None)

        # pass through first shared MLP
        x = self.bn1(F.relu(self.conv1(x)))
        x = self.bn2(F.relu(self.conv2(x)))

        # pass through second encoder layer
        x = self.tencoder2(x, mask=None)

        # pass through second MLP
        x = self.bn3(F.relu(self.conv3(x)))
        x = self.bn4(F.relu(self.conv4(x)))



        #pass through max pool
        x = self.max_pool(x)

        return x

